In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import sparse

In [3]:
from load_functions import *
from transform_functions import *
from score_functions import *
from i2i_functions import *

In [4]:
test_users = pd.read_csv('test_users.csv', dtype=str, names=['user_id'])
users_df = load_users('users.csv', test_users)
orgs_df = load_orgs('organisations.csv')
test_users_df = users_df[users_df.in_test==1][['user_id','city']]

In [5]:
reviews, user_agg, org_agg = load_reviews(users_df=users_df, orgs_df=orgs_df)
orgs_df = orgs_df.merge(org_agg, on='org_id')
users_df = users_df.merge(user_agg, on='user_id')

In [6]:
result, encs = prepare_reviews_i2i(reviews, users_df, orgs_df)

In [7]:
m=reviews_matrix(result, encs)

In [8]:
cc_mat = m.T * m

In [51]:
CC_2_J(cc_mat).max()

0.9990009990009991

In [47]:
CC_2_pmi(cc_mat).min()

-8.90067152152789

In [ ]:
ease_solution(cc_mat)

In [23]:
aspects = load_aspects()
rubrics = load_rubrics(orgs_df=orgs_df)
features = load_features(orgs_df=orgs_df)

In [7]:
user_enc = index_items(users_df['user_id'])
org_enc = index_items(orgs_df['org_id'])

aspects_enc = index_items(aspects['aspect_id'])
rubrics_enc = index_items(rubrics['rubric_id'])
features_enc = index_items(features['feature_id'])

In [16]:
users_df.head()

[autoreload of transform_functions failed: Traceback (most recent call last):
  File "/Users/rodrask/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/rodrask/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/rodrask/.pyenv/versions/3.9.5/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/rodrask/.pyenv/versions/3.9.5/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 851, in exec_module
  File "<frozen importlib._bootstrap_external>", line 988, in get_code
  File "<frozen importlib._bootstrap_external>", line 918, in source_to_code
  File "<frozen importlib._bootstrap>", line 2

,user_id,city,in_test,n_reviews,mean_score,mean_aspects,n_travels
0,523295021912509756,msk,0,3,5.000000,0.000000,1
1,11952159487361099606,msk,0,1,5.000000,0.000000,0
2,16879036589969590999,msk,0,3,4.333333,1.666667,1
3,12791716990148606332,msk,0,7,4.714286,0.428571,0
4,11642393216024958726,msk,0,3,4.000000,0.000000,0


In [42]:
users_df['n_travels'].describe()

count    1.252801e+06
mean     1.365293e-01
std      7.271824e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.060000e+02
Name: n_travels, dtype: float64

In [45]:
users_df[(~users_df.in_test) & (users_df.n_travels>0) & (users_df.n_reviews>5)][['user_id']]

[autoreload of transform_functions failed: Traceback (most recent call last):
  File "/Users/rodrask/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/rodrask/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/rodrask/.pyenv/versions/3.9.5/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/rodrask/.pyenv/versions/3.9.5/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 851, in exec_module
  File "<frozen importlib._bootstrap_external>", line 988, in get_code
  File "<frozen importlib._bootstrap_external>", line 918, in source_to_code
  File "<frozen importlib._bootstrap>", line 2

,user_id
5,7363061050701441135
22,2159172603395050629
46,3323865769999680988
51,2293090542920203967
73,3574805275042117179
...,...
1251928,10564769357210348127
1252096,15757565470439234757
1252274,15436939169422747487
1252335,12121515059175340126


In [11]:
prepare_test_users(test_users, user_enc, org_enc)

ValueError: y should be a 1d array, got an array of shape () instead.

In [20]:
reviews_mat = reviews_matrix(reviews[reviews.rating>=4.0], user_enc, org_enc)

In [21]:
CC_mat = reviews_mat.T * reviews_mat

In [22]:
l2=0.01
CC_mat += l2 * sparse.eye(CC_mat.shape[0])

In [ ]:
CC_mat = np.linalg.inv(CC_mat.todense())

In [109]:

preds = top_recs(orgs_df, test_users)

In [108]:
final_pred = top_recs(orgs_df, test_users)
save_predictions(final_pred)